# Intro
---
In this project, I generate fake lyrics for the Red Hot Chili Peppers using Keras and an LSTM RNN. Here is a quick read on this technique: https://towardsdatascience.com/recurrent-neural-networks-and-lstm-4b601dd822a5. I used BeautifulSoup to scrape Red Hot Chili Peppers lyrics from https://www.lyrics.com/ and the textgenrnn library for natural language generation.

## Imports

In [9]:
import pandas as pd
import numpy as np

#for web scraping
from bs4 import BeautifulSoup
import requests

#for natural language generation
import sys
from keras.models import Sequential
from keras.layers import LSTM, Activation, Flatten, Dropout, Dense, \
                         Embedding, TimeDistributed, CuDNNLSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from textgenrnn import textgenrnn
import os

import warnings
warnings.filterwarnings("ignore")

## Part 1: Web Scraping
---

In [10]:
url = 'https://www.lyrics.com/artist/Red%20Hot%20Chili%20Peppers'
r = requests.get(url)
soup = BeautifulSoup(markup = r.text, features = 'html.parser')

My process for scraping lyrics.com was to play around with the website HTML and use inspect element to find what class certain features belonged to. For example, I clicked on the album cover and saw that albums were in an h3 class titled "artist-album-label." I then scraped from these classes using BeautifulSoup:

*step 1*
![](images/album_select.png)

*step 2*
![](images/example_html.png)

In [11]:
albums = soup.find_all('h3', {'class': 'artist-album-label'})

In [12]:
for i in albums[:10]:
    print(i.text)

Uncensored
The Red Hot Chili Peppers [1984]
Freaky Styley [1985]
Freaky Styley [Bonus Tracks] [1985]
Freaky Styley [Japan Bonus Tracks] [1985]
Hollywood (Africa) [1985]
The Uplift Mofo Party Plan [1987]
The Uplift Mofo Party Plan [1987]
Uplift Mofo Party Plan [Japan Bonus Tracks] [1987]
Mother's Milk [1989]


In [13]:
songs = soup.find_all('td',{'class': 'tal qx'})

In [17]:
for i in songs[20:40]:
    print(i.text)

Baby Appeal
 
American Ghost Dance
3:44
Battle Ship
1:53
The Brothers Cup
3:27
Catholic School Girls Rule
1:55
Freaky Styley
3:39
Hollywood (Africa)
5:03
Jungle Man
4:09
Lovin' and Touchin'
:36
Sex Rap
1:54


Every other entry was the song duration (with some songs missing this data), so I found I could separate the titles and durations using list splicing:

In [15]:
#songs
for i in songs[:20:2]:
    print(i.text)

Blackeyed Blonde
Buckle Down
Green Heaven
Mommy, Where's Daddy?
Out in L.A.
Police Helicopter
Sex Rap [Anthony's Rap]
Thirty Dirty Birds
Yertle the Turtle
You Always Sing the Same


In [16]:
#durations
for i in songs[21:41:2]:
    print(i.text)

 
3:44
1:53
3:27
1:55
3:39
5:03
4:09
:36
1:54


Each song also had an embedded hypertext reference to the lyrics page:

In [19]:
songs[0].a.attrs['href']

'/lyric/529874/Red+Hot+Chili+Peppers/Blackeyed+Blonde'

I used all of this to create a dataframe with songs, time durations, and lyrics:

In [21]:
sng = []
tim = []
lyr = []
for i in songs[::2]:
    sng.append(i.text)
for i in songs[1::2]:
    tim.append(i.text)

base_url = 'https://www.lyrics.com/'
for i in range(len(songs)):
    #no link to lyrics
    if songs[i].a is None:
        pass
    else:
        lyr_url = base_url + songs[i].a.attrs['href']
        r = requests.get(lyr_url)
        soup = BeautifulSoup(r.text, 'html.parser')
        lyrics = soup.find('pre', {'id': 'lyric-body-text'}).text
        lyr.append(lyrics)
        
df = pd.DataFrame({'song': sng, 'time': tim, 'lyrics': lyr})

In [22]:
df.head()

,song,time,lyrics
0,Blackeyed Blonde,,Pumpin' the blood through the heart of New Orl...
1,Buckle Down,,Hah!\r\nOn the ice\r\nNo holdin'\r\nMy soul\r\...
2,Green Heaven,,"About this planet, there is something I know\r..."
3,"Mommy, Where's Daddy?",,"Mommy, where's daddy?\r\nMommy, where's daddy?..."
4,Out in L.A.,,We're all a bunch of brothers livin' in a cool...


## Part 2: Data Cleaning
---

In [24]:
df['lyrics'][0]

"Pumpin' the blood through the heart of New Orleans\r\nShe's the mystic heat of the Bourbon street dream\r\nShe's just made out of flesh and bones\r\n\r\nBut let me tell you little boy\r\nYou better leave her alone\r\nLeroy Brown thought he was bad too\r\n'Till she left him floatin' in the old bayou\r\n\r\nShe's the kind of girl\r\nShe's built like a bomb\r\nShe's the blackeyed blackeyed\r\nBlackeyed blond, get down!\r\n\r\nThat blackeyed beauty with the golden crotch\r\nFrench electric sex a cock shocking swamp fox\r\nHeaten queen of sleeze she's hot to box\r\n\r\nBut let me tell you little boy\r\nShe'll clean your clock\r\nA slick and sly spy\r\nStuck in the muck of the moat\r\n\r\nBlew his mind to find a diamond in the boat\r\nDouble-o-dooms day for Mr. James Bond\r\nShe's the blackeyed blackeyed\r\nBlackeyed blond, Get down!"

Each new line was separated by '\r\n' which is the newline character in Windows, so I replaced these with spaces:

In [25]:
def lyr_fixer(x):
    fixed = ''
    lyr_lst = x.split('\r\n')
    for item in lyr_lst[:-1]:
        fixed += item + ' '
    fixed += lyr_lst[-1]
    return fixed

In [26]:
df['lyrics'] = df['lyrics'].apply(lyr_fixer)

In [27]:
df['lyrics'][0]

"Pumpin' the blood through the heart of New Orleans She's the mystic heat of the Bourbon street dream She's just made out of flesh and bones  But let me tell you little boy You better leave her alone Leroy Brown thought he was bad too 'Till she left him floatin' in the old bayou  She's the kind of girl She's built like a bomb She's the blackeyed blackeyed Blackeyed blond, get down!  That blackeyed beauty with the golden crotch French electric sex a cock shocking swamp fox Heaten queen of sleeze she's hot to box  But let me tell you little boy She'll clean your clock A slick and sly spy Stuck in the muck of the moat  Blew his mind to find a diamond in the boat Double-o-dooms day for Mr. James Bond She's the blackeyed blackeyed Blackeyed blond, Get down!"

All better. Now I save my lyrics in a text file so I can use it in the LSTM RNN model:

In [28]:
lyrics = list(df['lyrics'])

In [29]:
with open('lyrics_text.txt','w',encoding='utf-8') as filehandle:
    for item in lyrics:
        filehandle.write('%s\n' % item)

## Part 3: Model Building
---
I ran into computing power problems here, since I wanted to create a more complex neural network. This model has 50 nodes, 4 layers, and 20 epochs. It took around 3 hours to generate on my laptop:

In [2]:
model_cfg = {
    'rnn_size': 50,
    'rnn_layers': 4,
    'rnn_bidirectional': True,
    'max_length': 15,
    'max_words': 10000,
    'dim_embeddings': 100,
    'word_level': False,
}
train_cfg = {
    'line_delimited': True,
    'num_epochs': 20,
    'gen_epochs': 25,
    'batch_size': 750,
    'train_size': 0.8,
    'dropout': 0.0,
    'max_gen_length': 300,
    'validation': True,
    'is_csv': False
}

In [3]:
model_name = '50nds_4Lrs_20epchs_Model'
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path='lyrics_text.txt',
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=train_cfg['batch_size'],
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    max_gen_length=train_cfg['max_gen_length'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=model_cfg['dim_embeddings'],
    word_level=model_cfg['word_level'])

602 texts collected.
Training new model w/ 4-layer, 50-cell Bidirectional LSTMs
Training on 225,043 character sequences.
Epoch 1/20
300/300 [==============================] - ETA: 50:17 - loss: 4.35 - ETA: 28:57 - loss: 4.23 - ETA: 21:47 - loss: 4.64 - ETA: 18:12 - loss: 4.89 - ETA: 16:12 - loss: 4.79 - ETA: 14:53 - loss: 4.56 - ETA: 13:48 - loss: 4.36 - ETA: 12:58 - loss: 4.20 - ETA: 12:20 - loss: 4.09 - ETA: 11:48 - loss: 3.99 - ETA: 11:23 - loss: 3.92 - ETA: 11:02 - loss: 3.86 - ETA: 10:45 - loss: 3.80 - ETA: 10:30 - loss: 3.74 - ETA: 10:15 - loss: 3.69 - ETA: 10:03 - loss: 3.65 - ETA: 9:58 - loss: 3.6182 - ETA: 9:51 - loss: 3.584 - ETA: 9:50 - loss: 3.554 - ETA: 9:42 - loss: 3.529 - ETA: 9:36 - loss: 3.507 - ETA: 9:30 - loss: 3.486 - ETA: 9:23 - loss: 3.468 - ETA: 9:17 - loss: 3.452 - ETA: 9:11 - loss: 3.436 - ETA: 9:05 - loss: 3.424 - ETA: 9:01 - loss: 3.409 - ETA: 8:59 - loss: 3.397 - ETA: 8:56 - loss: 3.385 - ETA: 8:51 - loss: 3.375 - ETA: 8:46 - loss: 3.364 - ETA: 8:41 - loss: 

300/300 [==============================] - ETA: 10:29 - loss: 2.10 - ETA: 10:22 - loss: 2.10 - ETA: 9:40 - loss: 2.1192 - ETA: 9:17 - loss: 2.094 - ETA: 8:56 - loss: 2.101 - ETA: 8:43 - loss: 2.102 - ETA: 8:33 - loss: 2.100 - ETA: 8:26 - loss: 2.100 - ETA: 8:21 - loss: 2.092 - ETA: 8:15 - loss: 2.086 - ETA: 8:12 - loss: 2.082 - ETA: 8:08 - loss: 2.083 - ETA: 8:05 - loss: 2.092 - ETA: 8:03 - loss: 2.088 - ETA: 8:00 - loss: 2.086 - ETA: 7:57 - loss: 2.087 - ETA: 7:54 - loss: 2.082 - ETA: 7:51 - loss: 2.082 - ETA: 7:48 - loss: 2.082 - ETA: 7:45 - loss: 2.079 - ETA: 7:42 - loss: 2.072 - ETA: 7:39 - loss: 2.070 - ETA: 7:38 - loss: 2.070 - ETA: 7:35 - loss: 2.069 - ETA: 7:32 - loss: 2.068 - ETA: 7:31 - loss: 2.069 - ETA: 7:29 - loss: 2.068 - ETA: 7:26 - loss: 2.063 - ETA: 7:23 - loss: 2.056 - ETA: 7:21 - loss: 2.053 - ETA: 7:19 - loss: 2.053 - ETA: 7:17 - loss: 2.050 - ETA: 7:15 - loss: 2.048 - ETA: 7:13 - loss: 2.047 - ETA: 7:11 - loss: 2.049 - ETA: 7:09 - loss: 2.048 - ETA: 7:07 - loss: 2.

300/300 [==============================] - ETA: 7:42 - loss: 1.511 - ETA: 7:39 - loss: 1.495 - ETA: 7:38 - loss: 1.510 - ETA: 7:38 - loss: 1.524 - ETA: 7:35 - loss: 1.532 - ETA: 7:33 - loss: 1.533 - ETA: 7:31 - loss: 1.538 - ETA: 7:31 - loss: 1.530 - ETA: 7:31 - loss: 1.542 - ETA: 7:29 - loss: 1.541 - ETA: 7:28 - loss: 1.538 - ETA: 7:26 - loss: 1.538 - ETA: 7:26 - loss: 1.542 - ETA: 7:27 - loss: 1.546 - ETA: 7:25 - loss: 1.550 - ETA: 7:23 - loss: 1.554 - ETA: 7:21 - loss: 1.557 - ETA: 7:20 - loss: 1.556 - ETA: 7:25 - loss: 1.554 - ETA: 7:46 - loss: 1.554 - ETA: 8:11 - loss: 1.551 - ETA: 8:16 - loss: 1.549 - ETA: 8:11 - loss: 1.546 - ETA: 8:07 - loss: 1.548 - ETA: 8:03 - loss: 1.549 - ETA: 7:59 - loss: 1.549 - ETA: 7:55 - loss: 1.549 - ETA: 7:52 - loss: 1.549 - ETA: 7:48 - loss: 1.551 - ETA: 7:45 - loss: 1.549 - ETA: 7:42 - loss: 1.551 - ETA: 7:39 - loss: 1.550 - ETA: 7:37 - loss: 1.550 - ETA: 7:34 - loss: 1.550 - ETA: 7:31 - loss: 1.548 - ETA: 7:28 - loss: 1.550 - ETA: 7:25 - loss: 1.5

300/300 [==============================] - ETA: 7:48 - loss: 1.368 - ETA: 7:42 - loss: 1.342 - ETA: 7:42 - loss: 1.345 - ETA: 7:42 - loss: 1.346 - ETA: 7:43 - loss: 1.349 - ETA: 7:40 - loss: 1.346 - ETA: 7:37 - loss: 1.363 - ETA: 7:35 - loss: 1.355 - ETA: 7:34 - loss: 1.352 - ETA: 7:33 - loss: 1.359 - ETA: 7:30 - loss: 1.359 - ETA: 7:28 - loss: 1.354 - ETA: 7:27 - loss: 1.350 - ETA: 7:26 - loss: 1.356 - ETA: 7:24 - loss: 1.356 - ETA: 7:22 - loss: 1.355 - ETA: 7:19 - loss: 1.353 - ETA: 7:18 - loss: 1.350 - ETA: 7:17 - loss: 1.351 - ETA: 7:15 - loss: 1.351 - ETA: 7:13 - loss: 1.352 - ETA: 7:13 - loss: 1.355 - ETA: 7:12 - loss: 1.354 - ETA: 7:11 - loss: 1.357 - ETA: 7:11 - loss: 1.361 - ETA: 7:11 - loss: 1.359 - ETA: 7:10 - loss: 1.357 - ETA: 7:09 - loss: 1.354 - ETA: 7:07 - loss: 1.355 - ETA: 7:05 - loss: 1.358 - ETA: 7:03 - loss: 1.359 - ETA: 7:02 - loss: 1.358 - ETA: 7:01 - loss: 1.357 - ETA: 6:59 - loss: 1.354 - ETA: 6:57 - loss: 1.351 - ETA: 6:55 - loss: 1.351 - ETA: 6:55 - loss: 1.3

300/300 [==============================] - ETA: 7:53 - loss: 1.185 - ETA: 7:39 - loss: 1.239 - ETA: 7:37 - loss: 1.257 - ETA: 7:36 - loss: 1.247 - ETA: 7:37 - loss: 1.238 - ETA: 7:35 - loss: 1.230 - ETA: 7:34 - loss: 1.232 - ETA: 7:33 - loss: 1.228 - ETA: 7:33 - loss: 1.232 - ETA: 7:32 - loss: 1.219 - ETA: 7:30 - loss: 1.215 - ETA: 7:30 - loss: 1.225 - ETA: 7:29 - loss: 1.226 - ETA: 7:28 - loss: 1.224 - ETA: 7:27 - loss: 1.223 - ETA: 7:25 - loss: 1.220 - ETA: 7:23 - loss: 1.225 - ETA: 7:22 - loss: 1.222 - ETA: 7:20 - loss: 1.224 - ETA: 7:18 - loss: 1.222 - ETA: 7:16 - loss: 1.222 - ETA: 7:15 - loss: 1.220 - ETA: 7:13 - loss: 1.223 - ETA: 7:12 - loss: 1.223 - ETA: 7:10 - loss: 1.219 - ETA: 7:08 - loss: 1.223 - ETA: 7:07 - loss: 1.223 - ETA: 7:05 - loss: 1.223 - ETA: 7:04 - loss: 1.224 - ETA: 7:02 - loss: 1.221 - ETA: 7:00 - loss: 1.222 - ETA: 6:58 - loss: 1.222 - ETA: 6:56 - loss: 1.222 - ETA: 6:55 - loss: 1.222 - ETA: 6:53 - loss: 1.223 - ETA: 6:52 - loss: 1.223 - ETA: 6:50 - loss: 1.2

300/300 [==============================] - ETA: 7:54 - loss: 1.195 - ETA: 7:48 - loss: 1.139 - ETA: 7:47 - loss: 1.123 - ETA: 7:53 - loss: 1.131 - ETA: 7:53 - loss: 1.135 - ETA: 7:52 - loss: 1.131 - ETA: 7:49 - loss: 1.139 - ETA: 7:46 - loss: 1.143 - ETA: 7:44 - loss: 1.135 - ETA: 7:44 - loss: 1.133 - ETA: 7:42 - loss: 1.134 - ETA: 7:39 - loss: 1.129 - ETA: 7:37 - loss: 1.127 - ETA: 7:36 - loss: 1.126 - ETA: 7:35 - loss: 1.130 - ETA: 7:32 - loss: 1.127 - ETA: 7:30 - loss: 1.129 - ETA: 7:28 - loss: 1.130 - ETA: 7:27 - loss: 1.132 - ETA: 7:28 - loss: 1.135 - ETA: 7:27 - loss: 1.133 - ETA: 7:25 - loss: 1.133 - ETA: 7:23 - loss: 1.134 - ETA: 7:21 - loss: 1.135 - ETA: 7:19 - loss: 1.136 - ETA: 7:17 - loss: 1.139 - ETA: 7:16 - loss: 1.139 - ETA: 7:15 - loss: 1.138 - ETA: 7:14 - loss: 1.140 - ETA: 7:12 - loss: 1.137 - ETA: 7:10 - loss: 1.134 - ETA: 7:08 - loss: 1.133 - ETA: 7:06 - loss: 1.130 - ETA: 7:05 - loss: 1.132 - ETA: 7:03 - loss: 1.133 - ETA: 7:01 - loss: 1.132 - ETA: 6:59 - loss: 1.1

300/300 [==============================] - ETA: 8:02 - loss: 1.056 - ETA: 7:56 - loss: 1.074 - ETA: 7:46 - loss: 1.051 - ETA: 7:43 - loss: 1.050 - ETA: 7:44 - loss: 1.050 - ETA: 7:44 - loss: 1.048 - ETA: 7:40 - loss: 1.044 - ETA: 7:38 - loss: 1.044 - ETA: 8:00 - loss: 1.031 - ETA: 8:49 - loss: 1.032 - ETA: 9:28 - loss: 1.032 - ETA: 9:25 - loss: 1.035 - ETA: 9:13 - loss: 1.030 - ETA: 9:03 - loss: 1.037 - ETA: 8:55 - loss: 1.037 - ETA: 8:48 - loss: 1.034 - ETA: 8:40 - loss: 1.034 - ETA: 8:34 - loss: 1.030 - ETA: 8:28 - loss: 1.033 - ETA: 8:23 - loss: 1.029 - ETA: 8:17 - loss: 1.031 - ETA: 8:13 - loss: 1.028 - ETA: 8:08 - loss: 1.029 - ETA: 8:04 - loss: 1.027 - ETA: 8:00 - loss: 1.025 - ETA: 7:56 - loss: 1.027 - ETA: 7:54 - loss: 1.028 - ETA: 7:50 - loss: 1.025 - ETA: 7:48 - loss: 1.024 - ETA: 7:45 - loss: 1.022 - ETA: 7:42 - loss: 1.023 - ETA: 7:39 - loss: 1.020 - ETA: 7:36 - loss: 1.021 - ETA: 7:33 - loss: 1.017 - ETA: 7:30 - loss: 1.016 - ETA: 7:27 - loss: 1.018 - ETA: 7:25 - loss: 1.0

300/300 [==============================] - ETA: 7:55 - loss: 0.902 - ETA: 7:49 - loss: 0.938 - ETA: 7:51 - loss: 0.933 - ETA: 7:49 - loss: 0.938 - ETA: 7:53 - loss: 0.915 - ETA: 7:50 - loss: 0.904 - ETA: 7:48 - loss: 0.914 - ETA: 7:45 - loss: 0.913 - ETA: 7:43 - loss: 0.915 - ETA: 7:39 - loss: 0.916 - ETA: 7:37 - loss: 0.922 - ETA: 7:35 - loss: 0.921 - ETA: 7:32 - loss: 0.924 - ETA: 7:30 - loss: 0.920 - ETA: 7:28 - loss: 0.915 - ETA: 7:27 - loss: 0.914 - ETA: 7:25 - loss: 0.914 - ETA: 7:23 - loss: 0.917 - ETA: 7:20 - loss: 0.917 - ETA: 7:19 - loss: 0.919 - ETA: 7:18 - loss: 0.919 - ETA: 7:17 - loss: 0.918 - ETA: 7:15 - loss: 0.917 - ETA: 7:13 - loss: 0.917 - ETA: 7:12 - loss: 0.918 - ETA: 7:10 - loss: 0.919 - ETA: 7:08 - loss: 0.923 - ETA: 7:07 - loss: 0.925 - ETA: 7:05 - loss: 0.926 - ETA: 7:04 - loss: 0.926 - ETA: 7:02 - loss: 0.928 - ETA: 7:00 - loss: 0.929 - ETA: 6:59 - loss: 0.929 - ETA: 6:59 - loss: 0.932 - ETA: 6:58 - loss: 0.933 - ETA: 6:57 - loss: 0.936 - ETA: 6:55 - loss: 0.9

300/300 [==============================] - ETA: 7:44 - loss: 0.847 - ETA: 7:42 - loss: 0.846 - ETA: 7:44 - loss: 0.841 - ETA: 7:40 - loss: 0.844 - ETA: 7:37 - loss: 0.844 - ETA: 7:34 - loss: 0.838 - ETA: 7:32 - loss: 0.855 - ETA: 7:30 - loss: 0.870 - ETA: 7:28 - loss: 0.867 - ETA: 7:27 - loss: 0.867 - ETA: 7:28 - loss: 0.871 - ETA: 7:28 - loss: 0.871 - ETA: 7:26 - loss: 0.870 - ETA: 7:24 - loss: 0.865 - ETA: 7:22 - loss: 0.863 - ETA: 7:21 - loss: 0.859 - ETA: 7:20 - loss: 0.858 - ETA: 7:18 - loss: 0.862 - ETA: 7:16 - loss: 0.863 - ETA: 7:14 - loss: 0.864 - ETA: 7:13 - loss: 0.864 - ETA: 7:11 - loss: 0.864 - ETA: 7:11 - loss: 0.864 - ETA: 7:10 - loss: 0.864 - ETA: 7:10 - loss: 0.864 - ETA: 7:09 - loss: 0.867 - ETA: 7:07 - loss: 0.868 - ETA: 7:05 - loss: 0.868 - ETA: 7:03 - loss: 0.869 - ETA: 7:02 - loss: 0.871 - ETA: 7:01 - loss: 0.868 - ETA: 6:59 - loss: 0.868 - ETA: 6:57 - loss: 0.867 - ETA: 6:55 - loss: 0.868 - ETA: 6:54 - loss: 0.868 - ETA: 7:02 - loss: 0.869 - ETA: 7:13 - loss: 0.8

300/300 [==============================] - ETA: 8:01 - loss: 0.869 - ETA: 8:00 - loss: 0.832 - ETA: 7:55 - loss: 0.846 - ETA: 7:51 - loss: 0.841 - ETA: 7:50 - loss: 0.834 - ETA: 7:47 - loss: 0.827 - ETA: 7:46 - loss: 0.832 - ETA: 7:45 - loss: 0.831 - ETA: 7:41 - loss: 0.819 - ETA: 7:39 - loss: 0.809 - ETA: 7:37 - loss: 0.805 - ETA: 7:35 - loss: 0.806 - ETA: 7:33 - loss: 0.813 - ETA: 7:31 - loss: 0.814 - ETA: 7:29 - loss: 0.814 - ETA: 7:27 - loss: 0.811 - ETA: 7:26 - loss: 0.814 - ETA: 7:24 - loss: 0.815 - ETA: 7:22 - loss: 0.818 - ETA: 7:21 - loss: 0.816 - ETA: 7:20 - loss: 0.818 - ETA: 7:18 - loss: 0.816 - ETA: 7:16 - loss: 0.818 - ETA: 7:14 - loss: 0.816 - ETA: 7:12 - loss: 0.817 - ETA: 7:10 - loss: 0.816 - ETA: 7:09 - loss: 0.816 - ETA: 7:07 - loss: 0.814 - ETA: 7:05 - loss: 0.815 - ETA: 7:04 - loss: 0.815 - ETA: 7:02 - loss: 0.814 - ETA: 7:01 - loss: 0.813 - ETA: 6:59 - loss: 0.813 - ETA: 6:57 - loss: 0.810 - ETA: 6:55 - loss: 0.810 - ETA: 6:54 - loss: 0.811 - ETA: 6:52 - loss: 0.8

In [4]:
textgen.model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 15)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 100)      7800        input[0][0]                      
__________________________________________________________________________________________________
rnn_1 (Bidirectional)           (None, 15, 100)      60400       embedding[0][0]                  
__________________________________________________________________________________________________
rnn_2 (Bidirectional)           (None, 15, 100)      60400       rnn_1[0][0]                      
__________________________________________________________________________________________________
rnn_3 (Bid

In [ ]:
textgen = textgenrnn(weights_path='50nds_4Lrs_20epchs_Model_weights.hdf5',
                       vocab_path='50nds_4Lrs_20epchs_Model_vocab.json',
                       config_path='50nds_4Lrs_20epchs_Model_config.json')

textgen.generate_samples(50)

####################
Temperature: 0.2
####################
I was not so  Sailan other day  I didn't resided the bridge of sing you will never be a baby breads with a monkey mind of my design My love is my love inside We are the way to feel  We are the ones that we make and a finger  The land of lands  Like a bright of sheed Jude, land on the moon by to tak



I could not alone  I don't believe it's bad Slit my throat it's all I ever  Tell me when it's time to be I said  I'm about to go  I don't believe it's bad Slit my throat it's all I ever  The earth this I got a feeling of the sun that he should be the sun  But it won't be too long  I'm falling on yo

I could not what it's time to swees the cover of another perfect wonder the chili  I love me down form your head  I've got to make it  The world when you don't ever wanna feel and the love of love for you   Cat from you and I'm in it from the brave  She's the sun my heart continues to say I'd be th

I could not all the way  I don't I d


I want to party on your pussy, baby I want to party on your pussy, baby I want to party on your pussy, baby I want to party on your pussy, baby I want to party on your pussy, baby I want to party on your pussy, baby I want to party on your pussy, baby I want to party on your pussy, baby I want to p

I feel you  I'm falling into grace with you I want to know more than I gotta take it on the other side  I don't believe it's bad Slit my throat it's all I ever  Traped to the party plane  This I know we're not alone  The one kids of a friend  The sun gets a bare in the street of love for the brothe

I could not little bit and really now  And we like the sea  The sun gets a side  I don't believe it's bad Slit my throat it's all I ever  Preaming of the create I make it day I'm falling all I ever  Every night I said yes what I want to do  What I don't you say lookin' for a nat  Nevermind the worl

I could not cheetah Long in the green suit  He's a freak of nature Walkin' to the baby and she m


I was not so  Sailated baby my aeroplane I give it a ride Something in the face of my desided And I don't want to say and goes to give it to your daughter the trees mountain   Hey oh  Oh oh (I steal it all the best  I can  I don't believe it's bad Slit my throat it's all I ever  Time on the other s



I was not so  Standing in line to be a slave  Show me your soul  Sentimental gentlemen are not afraid  Funny how the price gets paid  We pilli on your pussy, baby I want to party on your pussy, baby I want to party on your pussy, baby I want to party on your pussy, baby I want to party on your puss

I can't tell iff I'm a king or ear   The laws is really wanna  I saw you always stay me good  I've got to say I got a factory of  Fight unker flows that I want to be I do not at from free  I've said they take a little perfect of the earth One, two, three forsake Balling the hooran we'll never be my

I could be worlds that when they do in your strange of the earth One, two, three forst and the




I could be warm  What I've got you've got to give it to your daughter the treets of the card of my darling  A little pon that I can she got to find  We are the ones that we make it  You love me like you used to do?  When I don't want to say away your spirit upon your brother girl  I saw you what I 

I could not what to do  I don't believe it's bad Slit my throat it's all I ever  How long how long we love him come for the sun  The sun gets a state of mind  I'm a minor ground and a little face of the earth One, two, three forsake Balling better fuckin' the banana antide I don't believe it's bad 



What I've got you've got to give it to your daughter man  And it all in my ass  And that life we don't you see me getting high Knock me down I'm not bigger than life  If you see me getting high Knock me down I'm not do?  When I want to refuse  I mest for the universe I make the man  The sach of you

Give it away give it away now

I could not all the way  I don't I don't believe it's bad Sli




I was not so  Standing in line to be a funky crime  She scream deel I don't know what it all and  Think it's made there  You like to me alright  We just to say what I want to do  What is not to revent the sun  The sun gets a side  He's a repeat of my crarie  Make me alone for a face  She they stare



I can't tell iff I saw a little for a kiss I'm a double down down in your gamen   The morning of realing the world I know I know it's you You say hello and then I say I do  I am a jungle man I am a jungle man I get all the bush I can I am a jungle man I am a jungle man I am a jungle man I am a jung





I could not all the way (yeah yeah)  Oh oh (I said you be the back of beauty spiked to do  I'm pretolin' to be so the place  The party's the brothers cup) (We're the brothers cup) (We're the brothers cup) (We're the brothers cup) (We're the brothers cup) (We're the brothers cup coming dolphin sound

I want to party on your pussy, baby I want to party on your pussy, baby I want to party 


I could be worlds back I can't take me to the polle when you semal me We've got to make it  I take to know that this way to Have all me better girl  What I say inside of the earth And show you hear meet in the nouven hearts that I say I do what I want to do  What I've got you've got to give it to y



I want to party on your pussy, baby I want to party on your pussy, baby I want to party on your pussy, baby I want to party on your pussy, baby I want to party on your pussy, baby I want to party on your pussy, baby I want to party on your pussy, baby I want to party on your pussy, baby I want to p

I feel you  I want to party on your pussy, baby I want to party on your pussy, baby I want to party on your pussy, baby I want to party on your pussy, baby I want to party on your pussy I want to party on your pussy, baby I want to party on your pussy I want to party on your pussy, baby I want to p

The brothers cup) (We're the brothers cup) (We're the brothers cup) (We're the brothers cup) (


What I've got you've got to get it put it in you What I've got you've got to give it to your daughter blood  She's so selver and she's so strong to me Beneath the marquee   Cause on a little love for a banana in the street of your loney way I do what I want to do  What I do  I want to be your party



I don't believe it's bad Slit my throat it's all I ever  How long how long will I slide? Separate my side, I don't I don't believe it's bad Slit my throat it's all I ever  The earth we rock and she got a heart and the world wood so  This time of nothin'  The same of mind  Someone so much to say I'd

I could not what it's time to find  Blood soul  I can't tell iff I saw a little pump   I could not my love inside All the way the way (yeah yeah)  It's what I get swance  I gotta meter don't you like a lonlywood in the street of love and go on the face of my desided And I don't want it up and go se



Give it away now



## Extra Bonus Code
---
I realized that this code is reproducable for any artist on lyrics.com. This code allows you to name any singer and if their songs are listed on their artist page, it will return a dataframe with all of the artist's songs, song durations, and lyrics. Just copy this code and call lookup(). Enjoy!

In [30]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

def lookup():
    artist = input('Enter an artist name: ')
    url = 'https://www.lyrics.com/artist/' + artist.split(' ')[0].lower()
    for i in range(1,len(artist.split())):
        url += '%20' + artist.split(' ')[i].lower()
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    son = []
    tim = []
    songs = soup.find_all('td', {'class': 'tal qx'})
    for i in songs[::2]:
        s.append(i.text)
    for i in songs[1::2]:
        ti.append(i.text)
    base_url = 'https://www.lyrics.com/'
    lyr = []
    for i in range(len(songs)):
        if songs[i].a is None:
            pass
        else:
            lyr_url = base_url + songs[i].a.attrs['href']
            r = requests.get(lyr_url)
            soup = BeautifulSoup(r.text, 'html.parser')
            lyr.append(soup.find('pre', {'id': 'lyric-body-text'}).text)
    df = pd.DataFrame({'song': son, 'time': tim, 'lyrics': lyr})
    return df